On va tenter de simuler une pseudo carte scolaire. En effet, notre modélisation du lien entre taux de réussite au baccalauréat et revenu souffre probablement du fait qu'on ait associé le revenu des élèves à celui des communes de leur lycée. 
Or, des élèves viennent d'autres communes non pourvues d'établissement scolaire et le revenu médian peut être très hétérogène entre deux communes. 
On va donc essayer de refaire une carte scolaire de la manière suivante :
- Chaque commune sera associée à un point géographique. Si elle est dépourvue de lycée, on l'associera au lycée le plus proche. 
- On ponderera alors un revenu médian pour chaque lycée en fonction du revenu médian de chaque commune qui lui est associée et de la population des communes prises en compte. On posera donc l'hypothèse que la proportion de bacheliers potentiels est identique dans chaque commune (ce qui est discutable étant donné qu'il y'a une ségrégation spatiale générationnelle sur le territoire français)
- On procédera ainsi à une nouvelle régression avec les données obtenues

On doit donc créer une base de données adaptée. Il faut qu'on ait dedans
- Chaque établissement scolaire, avec son nombre de candidats et le taux de réussite par filières. 
- Chaque commune avec le revenu médian associé
- Les coordonnées géographiques des lycées et des communes 
En plus des bases de données utilisées précédemment, nous avons utilisé une base de données de La Poste pour avoir la longitude et la latitude de chaque commune. 

In [ ]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download
import seaborn as sns

  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-2movrial
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-2movrial


### A - Traitement de données pour le modèle

On va chercher à créer une base de données regroupant :
- L'ensemble des communes française, leur population et leur revenu médian 
- Les établissements scolaires avec leur taux de réussite dans chaque filière
- La situation géographique des villes et des lycées

Pour ce faire, nous utiliserons les bases précédemment mises en oeuvre dans la partie Manipulation afin d'avoir accès à la plupart de ces informations. Etant donné que la population n'était pas renseignée ainsi que la situation géographique de chaque commune, nous avons ajouté quelques bases de données pour obtenir ces bases de données supplémentaire

In [ ]:
#Importations des bases de données et début de nettoyage

#Base de communes
df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )

#Base des lycées
dflyc = pd.read_excel('base_def_SansDOMTOM.xlsx')
dflyc = dflyc[dflyc['Code_commune'].str.startswith('2B')==False] #On exclut la Corse pour faciliter des conversions de chaînes de caractère en nombre entier pour des jointures futures
dflyc = dflyc[dflyc['Code_commune'].str.startswith('2A')==False]

#Base des coordonnées géographiques des communes 
dfgeo = pd.read_csv('communes-departement-region.csv')

#Base population
dfpop = pd.read_excel('ensemble.xlsx', sheet_name=4, header=7)
dfpop =dfpop[["Code département", "Code commune", "Population totale"]]
dfpop = dfpop[dfpop['Code département'].str.startswith('2B')==False] #On exclut la Corse
dfpop = dfpop[dfpop['Code département'].str.startswith('2A')==False]
dfpop["Code département"] = dfpop["Code département"].astype(int)
for i in range(dfpop.shape[0]):
    dfpop.loc[dfpop.index[i], "Code département"] = dfpop.loc[dfpop.index[i], "Code département"]*1000 #On recrée les codes communes en les liant aux codes départementaux pour uniformiser avec les codes communes des autres bases afin que nos jointures fonctionnent
dfpop['Code Commune INSEE'] = dfpop['Code département'] + dfpop['Code commune'] #Les codes communes seront alors compatibles avec ceux de la base de données des communes
dfpop = dfpop.drop(["Code département", "Code commune"], axis=1)

In [ ]:
# Nettoyage de la base dfgeo
dfgeo=dfgeo[dfgeo['code_postal']<97000]
dfgeo = dfgeo.drop(["code_postal", "libelle_acheminement", "ligne_5", "nom_commune_complet", "nom_departement", "nom_region","article"], axis=1)
dfgeo

#Nettoyagede la base dflyc
dflyc.columns
dflyc = dflyc.drop(["Unnamed: 0", "Valeur ajoutée du taux de réussite_GNLE", "Valeur ajoutée du taux de réussite_Valeur ajoutée techno", "Nombre d'élèves présents au Bac_TOTAL_LGT","Taux de réussite bruts_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_Pro", "Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19", "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19", "coordonnee_X", "coordonnee_Y", "epsg"], axis=1)
dflyc

#Amélioration de la base concernant les communes françaises
df_com.columns = df_com.columns.map('_'.join).str.strip('_')
df_com = df_com.drop(["Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19","Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"], axis=1)
df_com = df_com[df_com["Code géographique_CODGEO"].str.startswith('2A')==False]
df_com = df_com[df_com["Code géographique_CODGEO"].str.startswith('2B')==False] #On enlève la Corse pour faciliter des manipulations futures
df_com["Code géographique_CODGEO"] = df_com["Code géographique_CODGEO"].astype(int) #Pour la jointure, on a besoin que CODGEO soit un entier
df_com2 = df_com.merge(dfpop, how='left', left_on = "Code géographique_CODGEO", right_on='Code Commune INSEE')
df_com2["Code géographique_CODGEO"] = df_com2["Code géographique_CODGEO"].astype(str) #Pour les jointures futures, on a besoin de chaines de carractères

In [ ]:
#Fusion des bases

#Fusion population/lycée
dflyc['Code_commune'] = dflyc['Code_commune'].astype(int) #On adapte les types pour faciliter les jointures
dflyc = dflyc.merge(dfpop, left_on = 'Code_commune', right_on='Code Commune INSEE')
dflyc['Code_commune'] = dflyc['Code_commune'].astype(str)

#Fusion commune/géographie
dfcomgeo  = df_com2.merge(dfgeo, how= 'left', left_on='Code géographique_CODGEO', right_on = 'code_commune_INSEE')
dfcomgeo = dfcomgeo.drop(["nom_commune_postal", "nom_commune"], axis=1)

#Fusion commune, lycée 
dfmodel = pd.merge(dflyc, dfcomgeo, how = 'outer', left_on = "Code_commune", right_on ='code_commune_INSEE')
dfmodel

Etant donné le nombre de colonnes que l'on a obtenu, on va chercher à limiter les doublons en regroupant certaines données dans des colonnes similaires. Pour les lycées, leur situation géographique sera alors associée à celle de leur commune mais on pose l'hypothèse que cela ne nuira pas considérablement à notre modèle, la carte scolaire étant assez aléatoirement respectée dans les grandes villes en raison de la diversité des filières et de stratégies familiales.

In [ ]:
dfmodelc = dfmodel.copy()

dfmodelc["Informations établissement_Ville"]= dfmodelc["Informations établissement_Ville"].fillna(0)
dfmodelcnz = dfmodelc[dfmodelc["Informations établissement_Ville"]!=0]
dfmodelcz = dfmodelc[dfmodelc["Informations établissement_Ville"]==0]
for i in range(dfmodelcnz.shape[0]):
    dfmodelcnz.loc[dfmodelcnz.index[i], "Libellé géographique_LIBGEO"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Informations établissement_Ville"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "code_commune_INSEE"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Code_commune"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "Médiane (€)_Q219_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Médiane (€)_Q219_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "latitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"latitude_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "longitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"longitude_x"]
dfmodelc = pd.concat([dfmodelcnz, dfmodelcz])

dfmodelc = dfmodelc.drop(["Code_commune","Informations établissement_Ville","code_commune", "latitude_x", "longitude_x", "Médiane (€)_Q219_x", "Code géographique_CODGEO", "Code Commune INSEE_x", 'Population totale_x', 'Code Commune INSEE_y'], axis=1)
dfmodelc = dfmodelc.drop_duplicates() 
dfmodelc = dfmodelc.reset_index()
dfmodelc

### B - Le modèle

On va essayer de déterminer, pour chaque commune dépourvue de lycée, le lycée qui est le plus proche géographiquement. Ainsi, pour chaque lycée, on va grouper en son sein des communes associées. 
On posera l'hypothèse que tous les élèves vont nécessairement être scolarisés dans un lycée de leur département pour alléger l'algorithme.

In [183]:
#On prépare quelques éléments pour faciliter les régressions futures
dfmodelc['Informations établissement_Académie'] = dfmodelc['Informations établissement_Académie'].fillna(0)
dfcomsanslyc = dfmodelc[dfmodelc['Informations établissement_Académie']==0] #La base de données des communes dépourvues de lycée
dfaveclyc = dfmodelc[dfmodelc['Informations établissement_Académie']!=0] #La base de données des lycées 

In [254]:
#Définition d'une distance
def distance(IDlycee, IDcommune, df):
    i = df.Identifiant_de_l_etablissement[df.Identifiant_de_l_etablissement==IDlycee].index.tolist()[0]
    j = dfcomsanslyc.code_commune_INSEE[dfcomsanslyc.code_commune_INSEE==IDcommune].index.tolist()[0]
    longcom = dfcomsanslyc.loc[dfcomsanslyc.index[j], 'longitude_y']
    latcom = dfcomsanslyc.loc[dfcomsanslyc.index[j], 'latitude_y']
    longlyc =df.loc[df.index[i], 'longitude_y']
    latlyc =df.loc[dfaveclyc.index[i], 'longitude_y']
    d = np.sqrt((longcom - longlyc)**2 + (latcom- latlyc)**2 )
    return d

In [250]:
'''obj= {}
for i in range(3):
    obj[df[i,"nometablisement"]] = [commune]
obj'''
i = dfaveclyc.Identifiant_de_l_etablissement[dfaveclyc.Identifiant_de_l_etablissement=='0040003G'].index.tolist()[0]
dfgt

,index,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Médiane (€)_Q219_y,Population totale_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
0,0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
1,1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
2,2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0
3,4,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.073766,6.181939,4,93.0
4,6,0040007L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PROFESSIONNEL BEAU DE ROCHAS,PU,NaN,NaN,NaN,NaN,104.0,87.0,DIGNE LES BAINS,19840,17240.0,4070,44.100068,6.234224,4,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,6640,0950656X,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL LE CORBUSIER,PU,NaN,NaN,NaN,NaN,79.0,75.0,CORMEILLES EN PARISIS,28810,24892.0,95176,48.978732,2.200460,95,11.0
3827,6641,0951618T,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL AUGUSTE ESCOFFIER,PU,NaN,NaN,NaN,NaN,157.0,76.0,ERAGNY SUR OISE,23720,18283.0,95218,49.022023,2.108030,95,11.0
3828,6642,0950949R,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL JEAN MERMOZ,PU,NaN,NaN,NaN,NaN,134.0,68.0,MONTSOULT,28890,3593.0,95430,49.067790,2.322650,95,11.0
3829,6643,0783533S,VERSAILLES,YVELINES,LYCEE CAMILLE CLAUDEL (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,197.0,77.0,MANTES LA VILLE,18140,20654.0,78362,48.963651,1.714310,78,11.0


On va pour chaque établissement créer une liste vide qui contiendra les communes affectées à cet établissement. Elles seront regroupées sous la forme d'un dictionnaire. Dans chaque liste se trouveront les communes dépourvues de lycée ainsi que les communes dans lesquelles se trouve le lycée.

In [198]:
#Création des listes pour chaque lycée comportant la commune où il se trouve

#On distingue les lycées généraux/technologiques des lycées professionnels
dfgt = dfaveclyc.copy()
dfgt = dfgt.fillna(0)
dfgt = dfgt[dfgt["Nombre d'élèves présents au Bac_TOTAL_Pro"]==0]
dfgt = dfgt.reset_index()

dfp = dfgt = dfaveclyc.copy()
dfp = dfp.fillna(0)
dfp = dfp[dfp["Nombre d'élèves présents au Bac_TOTAL_Pro"]!=0]
dfp = dfp.reset_index()

#Général et technologique
lyceesgt = {} 
for i in range(dfgt.shape[0]):
    lyceesgt[dfgt.loc[dfgt.index[i], 'Identifiant_de_l_etablissement']] = [dfgt.loc[dfgt.index[i], "code_commune_INSEE"]]
    
    
#Professionnel
lyceesp = {}
for i in range(dfp.shape[0]):
    lyceesp[dfp.loc[dfp.index[i], 'Identifiant_de_l_etablissement']] = [dfp.loc[dfp.index[i], "code_commune_INSEE"]]

In [255]:
#On remplit les listes 

for df in [dfgt, dfp]:
    if df.shape == dfgt.shape: #Pour différencier, les 2 sont de dimensions différentes et cela rend le booléen moins ambigu
        lyc = lyceesgt
    else:
        lyc = lyceesp
    for j in range(dfcomsanslyc.shape[0]):
        indcom = df.loc[df.index[j], 'code_commune_INSEE']
        a = 0
        b = 10000000000000000
        for k in range(df.shape[0]):
            if df.loc[df.index[j], 'code_departement'] == df.loc[df.index[k], 'code_departement']: #On allège en ce restreignant au département
                indlyc = str(df.loc[df.index[k], 'Identifiant_de_l_etablissement'])
                print(indlyc)
                a = distance(indcom,indlyc,df)
                if (b<a):
                    b = a
                    IDlyc = df.loc[df.index[k], 'Identifiant_de_l_etablissement']
        lyc[str(IDlyc)].append(indcom)

0040003G


IndexError: list index out of range

In [216]:
dfgt

,index,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Médiane (€)_Q219_y,Population totale_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
0,0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
1,1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
2,2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0
3,4,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.073766,6.181939,4,93.0
4,6,0040007L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PROFESSIONNEL BEAU DE ROCHAS,PU,NaN,NaN,NaN,NaN,104.0,87.0,DIGNE LES BAINS,19840,17240.0,4070,44.100068,6.234224,4,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,6640,0950656X,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL LE CORBUSIER,PU,NaN,NaN,NaN,NaN,79.0,75.0,CORMEILLES EN PARISIS,28810,24892.0,95176,48.978732,2.200460,95,11.0
3827,6641,0951618T,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL AUGUSTE ESCOFFIER,PU,NaN,NaN,NaN,NaN,157.0,76.0,ERAGNY SUR OISE,23720,18283.0,95218,49.022023,2.108030,95,11.0
3828,6642,0950949R,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL JEAN MERMOZ,PU,NaN,NaN,NaN,NaN,134.0,68.0,MONTSOULT,28890,3593.0,95430,49.067790,2.322650,95,11.0
3829,6643,0783533S,VERSAILLES,YVELINES,LYCEE CAMILLE CLAUDEL (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,197.0,77.0,MANTES LA VILLE,18140,20654.0,78362,48.963651,1.714310,78,11.0


On va chercher à estimer un revenu pour chaque lycée à partir des communes qui lui sont associées en pondérant par les effectifs de population des-dites communes.

In [146]:
dfaveclycc=dfaveclyc.copy()

revenu_nouveau =[]
for i in range(dfaveclycc.shape[0]):
    revenu_nouveau.append(0) 
dfaveclycc['Revenu Local'] = revenu_nouveau


for elem in lycees:
    revenu_nouveau = pop*revenu de chaque com / somme des pops
    dfaveclycc.loc['indice', 'Revenu Local']=revenu_nouveau

#On colle revenu  à dataframe
#Nouvelle régression

,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Médiane (€)_Q219_y,Population totale_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region,Revenu Local
0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0,0
1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0,0
2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0,0
3,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0,0
4,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.073766,6.181939,4,93.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,0950656X,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL LE CORBUSIER,PU,NaN,NaN,NaN,NaN,79.0,75.0,CORMEILLES EN PARISIS,28810,24892.0,95176,48.978732,2.200460,95,11.0,0
6641,0951618T,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL AUGUSTE ESCOFFIER,PU,NaN,NaN,NaN,NaN,157.0,76.0,ERAGNY SUR OISE,23720,18283.0,95218,49.022023,2.108030,95,11.0,0
6642,0950949R,VERSAILLES,VAL D'OISE,LYCEE PROFESSIONNEL JEAN MERMOZ,PU,NaN,NaN,NaN,NaN,134.0,68.0,MONTSOULT,28890,3593.0,95430,49.067790,2.322650,95,11.0,0
6643,0783533S,VERSAILLES,YVELINES,LYCEE CAMILLE CLAUDEL (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,197.0,77.0,MANTES LA VILLE,18140,20654.0,78362,48.963651,1.714310,78,11.0,0


Nous allons donc effectuer à nouveau les régressions faites précédemment mais en tenant compte du revenu local, c'est-à-dire un revenu médian pondéré par les communes alentours aux lycées quand celles-ci sont dépourvues d'établissement scolaire. On recouvre ainsi davantage la réalité économique des bacheliers potentiels. 

In [ ]:
#Régression pour la filière générale
df_reg_GL = dfgt[[ "Revenu Local", "Taux de réussite bruts_GNLE"]]
df_reg_GL = df_reg_GL.dropna(axis = 0, how = "any")
df_reg_GL

x = np.array(df_reg_GL[["Revenu Local"]])
y = np.array(df_reg_GL[["Taux de réussite bruts_GNLE"]])

x = x.reshape(-1,1)
y = y.reshape(-1,1)

regression_model = LinearRegression()
regression_model.fit(x,y)
y_predicted = regression_model.predict(x)


plt.scatter(x,y, s=1)
plt.plot(x, y_predicted, color='r')
plt.title ("Taux de réussite au baccalauréat général en fonction du revenu médian")
plt.xlabel ("Revenu médian local")
plt.ylabel ("Taux de réussite")
plt.show()

r2 = r2_score (x, y)
print("R² = ", r2)


In [ ]:
#Régression pour la filière technologique
df_reg_TL = dfgt[[ "Revenu Local", "Taux de réussite bruts_Taux brut techno"]]
df_reg_TL = df_reg_GL.dropna(axis = 0, how = "any")
df_reg_TL

x = np.array(df_reg_TL[["Revenu Local"]])
y = np.array(df_reg_TL[["Taux de réussite bruts_Taux brut techno"]])

x = x.reshape(-1,1)
y = y.reshape(-1,1)

regression_model = LinearRegression()
regression_model.fit(x,y)
y_predicted = regression_model.predict(x)


plt.scatter(x,y, s=1)
plt.plot(x, y_predicted, color='r')
plt.title ("Taux de réussite au baccalauréat général en fonction du revenu médian")
plt.xlabel ("Revenu médian local")
plt.ylabel ("Taux de réussite")
plt.show()

r2 = r2_score (x, y)
print("R² = ", r2)

In [ ]:
#Régression pour la filière professionnelle
df_reg_GL = dfp[[ "Revenu Local", "Taux de réussite bruts_GNLE"]]
df_reg_GL = df_reg_GL.dropna(axis = 0, how = "any")
df_reg_GL

x = np.array(df_reg_techno[["Revenu Local"]])
y = np.array(df_reg_techno[["Taux de réussite bruts_GNLE"]])

x = x.reshape(-1,1)
y = y.reshape(-1,1)

regression_model = LinearRegression()
regression_model.fit(x,y)
y_predicted = regression_model.predict(x)


plt.scatter(x,y, s=1)
plt.plot(x, y_predicted, color='r')
plt.title ("Taux de réussite au baccalauréat général en fonction du revenu médian")
plt.xlabel ("Revenu médian local")
plt.ylabel ("Taux de réussite")
plt.show()

r2 = r2_score (x, y)
print("R² = ", r2)